In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict 
import plotly
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, roc_auc_score
import re
import seaborn as sns

## Helper functions

In [2]:
def standardize(X_train, X_test):
    """Standardize the columns of X_train and X_test w.r.t. means and stds from X_train. """
    if ~(X_train.columns == X_test.columns).all():
        raise ValueError("X_train and X_test should have the columns in the same order.")
    colnames = X_train.columns
    sc = StandardScaler()
    # Standardize train and test sets (it produces numpy arrays)
    X_train_scaled = sc.fit_transform(X_train)
    X_test_scaled = sc.transform(X_test)

    # Add column names back to resulting numpy arrays
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=colnames)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=colnames)
    return X_train_scaled, X_test_scaled

## Load the datasets

In [3]:
df_hh = pd.read_stata('../ada-2020-project-milestone-p2-stima83/Data/PisoFirme_AEJPol-20070024_household.dta')
df_ind = pd.read_stata('../ada-2020-project-milestone-p2-stima83/Data/114542-V1/PisoFirme_AEJPol-20070024_individual.dta')

In [4]:
# We split the dataset into the census (2000) and the survey data (2005)
df_hh = df_hh.dropna()
index_census = df_hh.columns[df_hh.columns.str.startswith('C')]
df_census = df_hh[index_census].copy()
index_survey = df_hh.columns[df_hh.columns.str.startswith('S')]
df_survey = df_hh[index_survey].copy()

In [5]:
# Split into test and training set.
train_split = 0.7
train_samples = np.random.rand(df_hh.shape[0]) < train_split
df_census_train = df_census.loc[train_samples]
df_census_test = df_census.loc[~train_samples]
df_survey_train = df_survey[train_samples]
df_survey_test = df_survey[~train_samples]

## Predict different survey targets
We predict the different survey values from the census data from 2000. We create a data array of the results for the different models. We calculate and report the $R^2$ scores for the different targets of the survey data and the different models.
We use the different models:
- Linear models
    - Linear regression
    - Lasso
    - Ridge regression
    - Stochastic vector regressor
- Non linear models
    - DecisionTreeRegressor
    - GradientBoostingRegressor
    - MLPRegressor


In [6]:
def generate_prediction_df(df_census_train, df_census_test, df_survey_train, df_survey_test, selected_models):
    results = np.zeros((len(selected_models), len(df_survey_train.columns)))
    for i, model in enumerate(selected_models):
        for j, (column_name, column_data) in enumerate(df_survey_train.iteritems()):
            model.fit(df_census_train, column_data)
            results[i,j] = model.score(df_census_test, df_survey_test[column_name])
    string_selected_models = [re.match(r'([a-zA-Z]+)\(.+', str(model)).group(1) for model in selected_models]
    return pd.DataFrame(results, columns=df_survey_train.columns, index=string_selected_models)

def wrapper_generate_prediction_df(df_census, df_survey, selected_models, standardize_=True, train_split=0.7):
    train_samples = np.random.rand(df_hh.shape[0]) < train_split
    df_census_train = df_census.loc[train_samples]
    df_census_test = df_census.loc[~train_samples]
    df_survey_train = df_survey[train_samples]
    df_survey_test = df_survey[~train_samples]
    if standardize_:
        df_census_train, df_census_test = standardize(df_census_train, df_census_test) 
    df_results = generate_prediction_df(df_census_train, df_census_test, df_survey_train, df_survey_test, selected_models)
    return df_results

In [7]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

selected_models = [LinearRegression(), Lasso(max_iter=5000), Ridge(), SVR(), DecisionTreeRegressor(), GradientBoostingRegressor(), MLPRegressor(max_iter=2000, hidden_layer_sizes=(200,))]


We calculate the $R^2$ score with a standardized data set and without any standardization. Different algorithms improve in performance when standardizing and others have in general no need for standardization like the Gradient Boosting Regressor.

In [9]:
# With standardization
df_results_standardized = wrapper_generate_prediction_df(df_census, df_survey, selected_models, standardize_=True)
import plotly.express as px
fig = px.imshow(df_results_standardized)
fig.show()

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.



In [11]:
selected_models = [LinearRegression(), Lasso(max_iter=5000), Ridge(), SVR(), DecisionTreeRegressor(), GradientBoostingRegressor()]
# Without standardization
df_results = wrapper_generate_prediction_df(df_census, df_survey, selected_models, standardize_=False)
import plotly.express as px
fig = px.imshow(df_results)
fig.show()

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 8740057.626247406, tolerance: 2226536.8402949325

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 8251146659.292706, tolerance: 6609362.073838578

/Users/Max/miniconda3/envs/ada/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 582958.52480793, tolerance: 293296.135206587



In [12]:
selected_features = df_results.transpose()['GradientBoostingRegressor'].sort_values(ascending=False).iloc[:5].index

In [13]:
selected_features

Index(['S_garbage', 'S_instcement', 'S_shcementfloor', 'S_cementfloordin',
       'S_cementfloorbed'],
      dtype='object')

## Functions

In [14]:
def plot_feature_importance(df_survey_train, df_census_train, model, survey_targets):
    fig = go.Figure()
    for target in survey_targets:
        model.fit(df_census_train, df_survey_train[target])
        feature_importances = model.feature_importances_
        df_feature_importances = pd.DataFrame(feature_importances, index=df_census_train.columns, columns=['Feature importance']).sort_values('Feature importance', ascending=False)
        fig.add_trace(go.Bar(x = df_feature_importances.index, y=df_feature_importances['Feature importance']))
    fig.show()

### S_garbage
Def: Uses garbage collection service

In [25]:
df_hh['S_garbage'].value_counts()

1.0    1617
0.0     336
Name: S_garbage, dtype: int64

This is a binary value, which tells us if we use garbage collection. In this case, we can use a classification algorithm and see how easily we can predict this value.

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
def plot_AUC_curve(model, survey_targets):
    fig = go.Figure()
    for target in survey_targets:
        model.fit(df_census_train, df_survey_train[target])
        prediction = model.predict(df_census_test)
        fpr, tpr, thresholds = roc_curve(df_survey_test[target], prediction)
        auc_metric = roc_auc_score(df_survey_test[target], prediction)
    print(f'The Area Under ROC curve is {auc_metric}')
    label_dict = {'x':'False positive rate', 'y':'True positive rate'}
    fig = px.line(x=fpr, y=tpr, width=600, height=600, title=f'ROC curve for {target}', labels=label_dict)
    fig.show()
plot_AUC_curve(GradientBoostingRegressor(), ['S_garbage'])

The Area Under ROC curve is 0.8329640718562874


In [16]:
plot_feature_importance(df_survey_train, df_census_train, GradientBoostingRegressor(), ['S_garbage'])

### S_instcement
Def: Installation of cement floor

In [26]:
df_hh['S_instcement'].value_counts()

1.0    1381
0.0     572
Name: S_instcement, dtype: int64

In [27]:
plot_AUC_curve(GradientBoostingRegressor(), ['S_instcement'])

The Area Under ROC curve is 0.7663427905169063


In [28]:
plot_feature_importance(df_survey_train, df_census_train, GradientBoostingRegressor(), ['S_instcement'])

### S_cementfloordin
Def: Cement floor in dining room

In [29]:
df_hh['S_cementfloordin'].value_counts()

1.0    1589
0.0     364
Name: S_cementfloordin, dtype: int64

In [32]:
plot_AUC_curve(GradientBoostingRegressor(), ['S_cementfloordin'])

The Area Under ROC curve is 0.7094537815126051


In [33]:
plot_feature_importance(df_survey_train, df_census_train, GradientBoostingRegressor(), ['S_cementfloordin'])

### S_shcementfloor
Def: Share of rooms with cement floors

In [34]:
import plotly.figure_factory as ff
px.histogram(df_hh['S_shcementfloor'], ['S_shcementfloor'])

## S_logsell

In [35]:
px.histogram(df_hh['S_logsell'], ['S_logsell'])